In [1]:
import math

import torch_geometric.nn.norm.batch_norm

from helper import *
from gnn import GNN

The State of the nth node is expressed by 4 real scalars:

v_n -> the voltage at the node
delta_n -> the voltage angle at the node (relative to the slack bus)
p_n -> the net active power flowing into the node
q_n -> the net reactive power flowing into the node


The physical characteristics of the network are described by the power flow equations:

p = P(v, delta, W)
q = Q(v, delta, W)

-> Relate local net power generation with the global state
-> Depends on the topology W of the grid

Electrical grid => Weighted Graph

Nodes in the graph produce/consume power

Edges represent electrical connections between nodes

State Matrix X element of  R(N x 4) => graph signal with 4 features
    => Each row is the state of the corresponding Node

Adjacency Matrix A => sparse matrix to represent the connections of each node, element of R(N x N), Aij = 1 if node i is connected to node j


We use the GNN as a mode phi(X, A, H)

We want to imitate the OPF solution p*
-> We want to minimize a loss L over a dataset T = {{X, p*}}

Objective Function:min arg H of sum over T L(p*,phi(X, A, H)) and we use L = Mean Squared error

Once GNN model phi is trained, we do not need the costly p* from pandapower to make predictions

Input Data X - R^(Nx4): Uniformly sample p_ref and q_ref of each load L with P_L ~ Uniform(0.9 * p_ref, 1.1 * p_ref) and Q_L ~ Uniform(0.9 * q_ref, 1.1 * q_ref)

Pseudocode for X and y in supervised learning:
for each P_L and Q_L:
    Create X with sub-optimal DCOPF results
    Create y with Pandapower calculating p* ACOPF with IPOPT


Spatio-Temporal GNN -> superposition of a gnn with spatial info and a temporal layer (Temporal Conv,LSTM etc.) ??

Bus => Node in GNN








In [9]:
# get lists of simbench codes
all_simbench_codes = sb.collect_all_simbench_codes()
all_simbench_codes

['1-complete_data-mixed-all-0-sw',
 '1-complete_data-mixed-all-1-sw',
 '1-complete_data-mixed-all-2-sw',
 '1-EHVHVMVLV-mixed-all-0-sw',
 '1-EHVHVMVLV-mixed-all-1-sw',
 '1-EHVHVMVLV-mixed-all-2-sw',
 '1-EHVHV-mixed-all-0-sw',
 '1-EHVHV-mixed-all-0-no_sw',
 '1-EHVHV-mixed-all-1-sw',
 '1-EHVHV-mixed-all-1-no_sw',
 '1-EHVHV-mixed-all-2-sw',
 '1-EHVHV-mixed-all-2-no_sw',
 '1-EHVHV-mixed-1-0-sw',
 '1-EHVHV-mixed-1-0-no_sw',
 '1-EHVHV-mixed-1-1-sw',
 '1-EHVHV-mixed-1-1-no_sw',
 '1-EHVHV-mixed-1-2-sw',
 '1-EHVHV-mixed-1-2-no_sw',
 '1-EHVHV-mixed-2-0-sw',
 '1-EHVHV-mixed-2-0-no_sw',
 '1-EHVHV-mixed-2-1-sw',
 '1-EHVHV-mixed-2-1-no_sw',
 '1-EHVHV-mixed-2-2-sw',
 '1-EHVHV-mixed-2-2-no_sw',
 '1-EHV-mixed--0-sw',
 '1-EHV-mixed--0-no_sw',
 '1-EHV-mixed--1-sw',
 '1-EHV-mixed--1-no_sw',
 '1-EHV-mixed--2-sw',
 '1-EHV-mixed--2-no_sw',
 '1-HVMV-mixed-all-0-sw',
 '1-HVMV-mixed-all-0-no_sw',
 '1-HVMV-mixed-all-1-sw',
 '1-HVMV-mixed-all-1-no_sw',
 '1-HVMV-mixed-all-2-sw',
 '1-HVMV-mixed-all-2-no_sw',
 '1-HVM

In [2]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')#'1-HV-mixed--0-no_sw'

In [19]:
train_data, val_data, test_data = read_unsupervised_dataset('1-HV-mixed--0-no_sw')

Extracting Node Types for the grid 1-HV-mixed--0-no_sw..
Extracting Edge Index and Edge Attributes for each Node Type for the grid 1-HV-mixed--0-no_sw
Extracting Node Types and Index Mapper..
Extracting Edge Index and Edge Attributes..
Adding Self Loops and Converting Edges to Undirected..
Hetero Data Created.
Reading all of the .csv files from the directory of 1-HV-mixed--0-no_sw ...
Processing Training Data for 1-HV-mixed--0-no_sw ...
Processing Validation Data for 1-HV-mixed--0-no_sw ...
Processing Test Data for 1-HV-mixed--0-no_sw ...
Processing complete.


In [20]:
#train_data[0].has_isolated_nodes()
#train_data[0].has_self_loops()
#train_data[0].is_undirected()
#train_data[0] = T.ToUndirected()(train_data[0])
train_data[0]

HeteroData(
  SB={ x=[1, 4] },
  PQ={ x=[20, 4] },
  PV={ x=[35, 4] },
  NB={ x=[8, 4] },
  (SB, -, PV)={
    edge_index=[2, 5],
    edge_attr=[5, 2]
  },
  (SB, -, PQ)={
    edge_index=[2, 9],
    edge_attr=[9, 2]
  },
  (SB, -, NB)={
    edge_index=[2, 0],
    edge_attr=[0]
  },
  (PV, -, PQ)={
    edge_index=[2, 46],
    edge_attr=[46, 2]
  },
  (PV, -, NB)={
    edge_index=[2, 6],
    edge_attr=[6, 2]
  },
  (PV, -, PV)={
    edge_index=[2, 66],
    edge_attr=[66, 2]
  },
  (PQ, -, NB)={
    edge_index=[2, 9],
    edge_attr=[9, 2]
  },
  (PQ, -, PQ)={
    edge_index=[2, 17],
    edge_attr=[17, 2]
  },
  (NB, -, NB)={
    edge_index=[2, 6],
    edge_attr=[6, 2]
  }
)

In [58]:
from math import tanh
x_dict = train_data[0].node_items()
x_dict

[('SB', {'x': tensor([[ 1.0000, 12.6063,  3.2692,  1.2376]])}),
 ('PQ',
  {'x': tensor([[  1.0000,  22.6509,   2.8762,   1.0764],
          [  1.0000,  12.6015,   5.1048,  13.4765],
          [  1.0000,  14.9366, -13.2957,   6.4814],
          [  1.0000,  12.3377,  31.1979,  19.7155],
          [  1.0000,  18.3740, -22.0108,   7.3980],
          [  1.0000,  18.0395,   3.1664,   1.2739],
          [  1.0000,  27.7888,   3.2053,   1.1336],
          [  1.0000,  26.0177, -13.5467,   7.2488],
          [  1.0000,  15.1376,  37.1663,  18.1150],
          [  1.0000,  21.2608,  -1.5991,  20.0734],
          [  1.0000,  18.5762,   2.8021,   1.2590],
          [  1.0000,  12.7370,   2.9836,   1.1099],
          [  1.0000,  28.0231,   2.9601,   1.1872],
          [  1.0000,  12.3896, -14.2209,   6.4525],
          [  1.0000,  15.6497,   5.0258,  13.5519],
          [  1.0000,  30.7598,   6.7317,  13.1374],
          [  1.0000,  27.8114,   2.9651,   1.1768],
          [  1.0000,  12.6149,   2.816

In [ ]:
grids_available = []
for nw_name in all_simbench_codes:
        net = sb.get_simbench_net(nw_name)
        print("Trying Network named " + nw_name + "...")

        #dict_probs = pp.diagnostic(net,report_style='None')
        #for bus_num in dict_probs['multiple_voltage_controlling_elements_per_bus']['buses_with_gens_and_ext_grids']:
        #    net.gen = net.gen.drop(net.gen[net.gen.bus == bus_num].index)

        #OPERATIONAL CONSTRAINTS

        #Set upper and lower limits of active-reactive powers of loads
        min_p_mw_val, max_p_mw_val, min_q_mvar_val, max_q_mvar_val = [], [], [], []
        p_mw = list(net.load.p_mw.values)
        q_mvar = list(net.load.q_mvar.values)

        for i in range(len(p_mw)):
            min_p_mw_val.append(p_mw[i])
            max_p_mw_val.append(p_mw[i])
            min_q_mvar_val.append(q_mvar[i])
            max_q_mvar_val.append(q_mvar[i])

        net.load.min_p_mw = min_p_mw_val
        net.load.max_p_mw = max_p_mw_val
        net.load.min_q_mvar = min_q_mvar_val
        net.load.max_q_mvar = max_q_mvar_val

        #Replace all ext_grids but the first one with generators and set the generators to slack= false
        ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
        pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

        #TODO: reactive power limits for gens?


        #TODO: reactive power limits for sgens?



        #NETWORK CONSTRAINTS

        #Maximize the branch limits

        #max_i_ka = list(net.line.max_i_ka.values)

        #for i in range(len(max_i_ka)):
        # max_i_ka[i] = max(max_i_ka)



        #Maximize line loading percents
        max_loading_percent = list(net.line.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.line.max_loading_percent = max_loading_percent

        #Maximize trafo loading percent
        max_loading_percent = list(net.trafo.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.trafo.max_loading_percent = max_loading_percent

        #Maximize trafo3w loading percent
        max_loading_percent = list(net.trafo3w.max_loading_percent.values)
        for i in range(len(max_loading_percent)):
            max_loading_percent[i] = 100.0
        net.trafo3w.max_loading_percent = max_loading_percent

        #Cost assignment
        pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
        pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
        pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

        try:
            pp.runpm_dc_opf(net) # Run DCOPP
        except pp.OPFNotConverged:
            text = "DC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ".SKIPPING THIS DATASET."
            print(text)
            continue
        print("GRID NAMED "+nw_name+" CONVERGES FOR DCOPF")
        grids_available.append(nw_name)


In [158]:
for nw_name in grids_available:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) # dcopp on all grids directly

1-HVMV-mixed-all-0-sw: Number of Buses = 1942
1-HVMV-mixed-all-0-no_sw: Number of Buses = 1665
1-HVMV-mixed-1.105-0-sw: Number of Buses = 401
1-HVMV-mixed-1.105-0-no_sw: Number of Buses = 158
1-HVMV-mixed-2.102-0-sw: Number of Buses = 421
1-HVMV-mixed-2.102-0-no_sw: Number of Buses = 178
1-HVMV-mixed-4.101-0-sw: Number of Buses = 411
1-HVMV-mixed-4.101-0-no_sw: Number of Buses = 166
1-HVMV-urban-all-0-sw: Number of Buses = 1791
1-HVMV-urban-all-0-no_sw: Number of Buses = 1470
1-HVMV-urban-2.203-0-sw: Number of Buses = 487
1-HVMV-urban-2.203-0-no_sw: Number of Buses = 196
1-HVMV-urban-3.201-0-sw: Number of Buses = 514
1-HVMV-urban-3.201-0-no_sw: Number of Buses = 217
1-HVMV-urban-4.201-0-sw: Number of Buses = 477
1-HVMV-urban-4.201-0-no_sw: Number of Buses = 184
1-HV-mixed--0-sw: Number of Buses = 306
1-HV-mixed--0-no_sw: Number of Buses = 64
1-HV-urban--0-sw: Number of Buses = 372
1-HV-urban--0-no_sw: Number of Buses = 82
1-MVLV-rural-all-0-sw: Number of Buses = 5479
1-MVLV-rural-all-0

In [ ]:
grids_ready = []
for nw_name in all_simbench_codes[6:]:
    net = sb.get_simbench_net(nw_name)
    print("Trying Network named " + nw_name + "...")

    #OPERATIONAL CONSTRAINTS

    #Set upper and lower limits of active-reactive powers of loads
    min_p_mw_val, max_p_mw_val, min_q_mvar_val, max_q_mvar_val = [], [], [], []
    p_mw = list(net.load.p_mw.values)
    q_mvar = list(net.load.q_mvar.values)

    for i in range(len(p_mw)):
        min_p_mw_val.append(p_mw[i])
        max_p_mw_val.append(p_mw[i])
        min_q_mvar_val.append(q_mvar[i])
        max_q_mvar_val.append(q_mvar[i])

    net.load.min_p_mw = min_p_mw_val
    net.load.max_p_mw = max_p_mw_val
    net.load.min_q_mvar = min_q_mvar_val
    net.load.max_q_mvar = max_q_mvar_val

    #Replace all ext_grids but the first one with generators and set the generators to slack= false
    ext_grids = [i for i in range(1,len(net.ext_grid.name.values))]
    pp.replace_ext_grid_by_gen(net,ext_grids=ext_grids, slack=False)

    #TODO: reactive power limits for gens?


    #TODO: reactive power limits for sgens?



    #NETWORK CONSTRAINTS

    #Maximize the branch limits

    #max_i_ka = list(net.line.max_i_ka.values)

    #for i in range(len(max_i_ka)):
    # max_i_ka[i] = max(max_i_ka)



    #Maximize line loading percents
    max_loading_percent = list(net.line.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.line.max_loading_percent = max_loading_percent

    #Maximize trafo loading percent
    max_loading_percent = list(net.trafo.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.trafo.max_loading_percent = max_loading_percent

    #Maximize trafo3w loading percent
    max_loading_percent = list(net.trafo3w.max_loading_percent.values)
    for i in range(len(max_loading_percent)):
        max_loading_percent[i] = 100.0
    net.trafo3w.max_loading_percent = max_loading_percent

    #Cost assignment
    pp.create_pwl_costs(net, [i for i in range(len(net.gen.name.values))],et="gen", points=[[[0, 20, 1], [20, 30, 2]] for _ in range(len(net.gen.name.values))])
    pp.create_pwl_costs(net, [i for i in range(len(net.sgen.name.values))],et="sgen", points=[[[0, 20, 0.25], [20, 30, 0.5]] for _ in range(len(net.sgen.name.values))])
    pp.create_pwl_costs(net, [i for i in range(len(net.ext_grid.name.values))],et="ext_grid", points=[[[0, 20, 2], [20, 30, 5]] for _ in range(len(net.ext_grid.name.values))])

    #ac_converged = True

    #start_vec_name = ""
    #for init in ["pf", "flat", "results"]:
    #    try:
    #        pp.runopp(net, init=init)  # Calculate ACOPF with IPFOPT
    #    except pp.OPFNotConverged:
    #        if init == "results":
    #            text = "AC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ". SKIPPING THIS GRID."
    #            print(text)
    #            break
    #        continue
    #    start_vec_name = init
    #    ac_converged = True
    #    break
    #if ac_converged:
    #    print("GRID NAMED "+nw_name+" CONVERGES FOR ACOPF" + "WITH THE START VECTOR OPTION " + start_vec_name + ".")


    try:
        pp.runpm_ac_opf(net) # Run DCOPP
    except pp.OPFNotConverged:
        text = "AC OPTIMAL POWERFLOW COMPUTATION DID NOT CONVERGE FOR NETWORK " + nw_name + ".SKIPPING THIS DATASET."
        print(text)
        continue
    print("GRID NAMED "+nw_name+" CONVERGES FOR ACOPF" + "WITH THE START VECTOR OPTION " + ".")
    grids_ready.append(nw_name)


In [4]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #julia acopf on all grids directly

1-HV-mixed--0-sw: Number of Buses = 306
1-HV-mixed--0-no_sw: Number of Buses = 64
1-HV-urban--0-sw: Number of Buses = 372
1-HV-urban--0-no_sw: Number of Buses = 82


In [6]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #pp acopf on all grids directly

1-HV-mixed--0-sw: Number of Buses = 306
1-HV-mixed--0-no_sw: Number of Buses = 64
1-HV-mixed--1-sw: Number of Buses = 355
1-HV-mixed--1-no_sw: Number of Buses = 94
1-HV-urban--0-sw: Number of Buses = 372
1-HV-urban--0-no_sw: Number of Buses = 82
1-HV-urban--1-sw: Number of Buses = 402
1-HV-urban--1-no_sw: Number of Buses = 100
1-MV-semiurb--0-sw: Number of Buses = 117
1-MV-semiurb--0-no_sw: Number of Buses = 115
1-MV-comm--0-sw: Number of Buses = 107
1-MV-comm--0-no_sw: Number of Buses = 103


In [163]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #julia acopf

1-HV-mixed--0-sw: Number of Buses = 306
1-HV-mixed--0-no_sw: Number of Buses = 64
1-HV-urban--0-sw: Number of Buses = 372
1-HV-urban--0-no_sw: Number of Buses = 82


In [161]:
for nw_name in grids_ready:
    net = sb.get_simbench_net(nw_name)
    print(nw_name + ": Number of Buses = " + str(len(net.bus))) #pp acopf

1-HV-mixed--0-sw: Number of Buses = 306
1-HV-mixed--0-no_sw: Number of Buses = 64
1-HV-urban--0-sw: Number of Buses = 372
1-HV-urban--0-no_sw: Number of Buses = 82
1-MV-semiurb--0-sw: Number of Buses = 117
1-MV-semiurb--0-no_sw: Number of Buses = 115
1-MV-comm--0-sw: Number of Buses = 107
1-MV-comm--0-no_sw: Number of Buses = 103


In [9]:
net = sb.get_simbench_net('1-HV-mixed--0-no_sw')


In [6]:
dcopf_acopf_available_grid_names = ["1-HV-mixed--0-no_sw","1-HV-urban--0-no_sw", "1-MV-comm--0-no_sw", "1-MV-semiurb--0-no_sw"]

In [7]:
datasets_df_as_list = []
for grid_name in dcopf_acopf_available_grid_names:
    print("Generating datasets for grid " + grid_name)
    while len(datasets_df_as_list) != 100:
        net = sb.get_simbench_net(grid_name)
        df = create_dataset_from_dcopf_and_acopf(net)
        if df is not None:
            datasets_df_as_list.append(df)
            print(str(len(datasets_df_as_list)) + " Dataset(s) Generated" + " for the grid " + grid_name)

    print("Saving the datasets for grid " + grid_name)
    path = os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Supervised\\Training\\" + grid_name
    if not os.path.exists(path):
        os.makedirs(path)

    for i in range(0, len(datasets_df_as_list)):
        newpath = path + "\\Dataset-" + str(i) + ".csv"
        datasets_df_as_list[i].to_csv(newpath)

    datasets_df_as_list.clear()

Generating datasets for grid 1-HV-mixed--0-no_sw
0 Datasets Generated for the grid 1-HV-mixed--0-no_sw
1 Datasets Generated for the grid 1-HV-mixed--0-no_sw
2 Datasets Generated for the grid 1-HV-mixed--0-no_sw
3 Datasets Generated for the grid 1-HV-mixed--0-no_sw
4 Datasets Generated for the grid 1-HV-mixed--0-no_sw
5 Datasets Generated for the grid 1-HV-mixed--0-no_sw
6 Datasets Generated for the grid 1-HV-mixed--0-no_sw
7 Datasets Generated for the grid 1-HV-mixed--0-no_sw
8 Datasets Generated for the grid 1-HV-mixed--0-no_sw
9 Datasets Generated for the grid 1-HV-mixed--0-no_sw
10 Datasets Generated for the grid 1-HV-mixed--0-no_sw
11 Datasets Generated for the grid 1-HV-mixed--0-no_sw
12 Datasets Generated for the grid 1-HV-mixed--0-no_sw
13 Datasets Generated for the grid 1-HV-mixed--0-no_sw
14 Datasets Generated for the grid 1-HV-mixed--0-no_sw
15 Datasets Generated for the grid 1-HV-mixed--0-no_sw
16 Datasets Generated for the grid 1-HV-mixed--0-no_sw
17 Datasets Generated for 

In [15]:
dcopf_available_grid_names = ["1-HVMV-mixed-all-0-no_sw", "1-HVMV-mixed-1.105-0-no_sw", "1-HVMV-mixed-2.102-0-no_sw","1-HVMV-mixed-4.101-0-no_sw", "1-HVMV-urban-all-0-no_sw", "1-HVMV-urban-2.203-0-no_sw", "1-HVMV-urban-3.201-0-no_sw", "1-HVMV-urban-4.201-0-no_sw", "1-HV-mixed--0-no_sw", "1-HV-urban--0-no_sw", "1-MVLV-rural-all-0-no_sw"]

In [16]:
datasets_df_as_list = []
for grid_name in dcopf_available_grid_names:
    print("Generating datasets for grid " + grid_name)
    while len(datasets_df_as_list) != 100:
        net = sb.get_simbench_net(grid_name)
        df = create_dataset_from_dcopf(net)
        if df is not None:
            datasets_df_as_list.append(df)
            print(str(len(datasets_df_as_list)) + " Dataset(s) Generated" + " for the grid " + grid_name)

    print("Saving the datasets for grid " + grid_name)
    path = os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Unsupervised\\Training\\" + grid_name
    if not os.path.exists(path):
        os.makedirs(path)

    for i in range(0, len(datasets_df_as_list)):
        newpath = path + "\\Dataset-" + str(i) + ".csv"
        datasets_df_as_list[i].to_csv(newpath)

    datasets_df_as_list.clear()

Generating datasets for grid 1-HVMV-mixed-all-0-no_sw
1 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
2 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
3 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
4 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
5 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
6 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
7 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
8 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
9 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
10 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
11 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
12 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
13 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
14 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
15 Dataset(s) Generated for the grid 1-HVMV-mixed-all-0-no_sw
16 Dataset(s) Generated f

In this revised version of the compute_node_embeddings function, the node features are weighted by the reverse admittance values in the adjacency matrix before they are summed to compute the node embeddings. The resulting node embeddings will reflect the strength of the connections between the nodes.

To use the reverse admittance values as the edge weights, you would need to pass the Ybus matrix as the adjacency matrix when calling the compute_node_embeddings function. The Ybus matrix should be converted to a PyTorch tensor before passing it to the function.

use the reverse admittance values as edge weights, you can modify the computation of the node embeddings to weight the node features by the reverse admittance values.

# Define the node types
node_types = ['Slack Node', 'Generator Node', 'Load Node']

# Define the number of nodes of each type in the graph
num_nodes = {
    'Slack Node': 1,
    'Generator Node': 20,
    'Load Node': 99
}


In [2]:
grid_names = [_ for _ in os.listdir(os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\data\\Supervised\\")]


In [3]:
graphdata_lst = read_multiple_supervised_datasets(grid_names)

Calculating edge index and edge attributes for the grid 1-HV-mixed--0-no_sw ...
Reading all of the .csv files from the directory of 1-HV-mixed--0-no_sw ...
Processing Training Data for 1-HV-mixed--0-no_sw ...
Processing Validation Data for 1-HV-mixed--0-no_sw ...
Processing Test Data for 1-HV-mixed--0-no_sw ...
Processing complete.
Calculating edge index and edge attributes for the grid 1-HV-urban--0-no_sw ...
Reading all of the .csv files from the directory of 1-HV-urban--0-no_sw ...
Processing Training Data for 1-HV-urban--0-no_sw ...
Processing Validation Data for 1-HV-urban--0-no_sw ...
Processing Test Data for 1-HV-urban--0-no_sw ...
Processing complete.
Calculating edge index and edge attributes for the grid 1-MV-comm--0-no_sw ...
Reading all of the .csv files from the directory of 1-MV-comm--0-no_sw ...
Processing Training Data for 1-MV-comm--0-no_sw ...
Processing Validation Data for 1-MV-comm--0-no_sw ...
Processing Test Data for 1-MV-comm--0-no_sw ...
Processing complete.
Cal

In [4]:
in_channels = 4
hidden_channels = 256
out_channels = 4
activation = "elu"
scaler = StandardScaler()
num_layers = 5
dropout = 0.0
jk = "last"
lr = 0.00001
layer_type = "TransConv"
# torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(in_channels, hidden_channels, num_layers, out_channels, dropout=dropout, norm=torch_geometric.nn.norm.batch_norm.BatchNorm(hidden_channels),jk=jk,layer_type=layer_type, activation=activation)#.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for name, param in model.named_parameters():
  print(name)
  print(param.size())

param = sum(p.numel() for p in model.parameters() if p.requires_grad)
param

bias
torch.Size([])
convs.0.lin_key.weight
torch.Size([256, 4])
convs.0.lin_key.bias
torch.Size([256])
convs.0.lin_query.weight
torch.Size([256, 4])
convs.0.lin_query.bias
torch.Size([256])
convs.0.lin_value.weight
torch.Size([256, 4])
convs.0.lin_value.bias
torch.Size([256])
convs.0.lin_edge.weight
torch.Size([256, 2])
convs.0.lin_skip.weight
torch.Size([256, 4])
convs.0.lin_skip.bias
torch.Size([256])
convs.1.lin_key.weight
torch.Size([256, 256])
convs.1.lin_key.bias
torch.Size([256])
convs.1.lin_query.weight
torch.Size([256, 256])
convs.1.lin_query.bias
torch.Size([256])
convs.1.lin_value.weight
torch.Size([256, 256])
convs.1.lin_value.bias
torch.Size([256])
convs.1.lin_edge.weight
torch.Size([256, 2])
convs.1.lin_skip.weight
torch.Size([256, 256])
convs.1.lin_skip.bias
torch.Size([256])
convs.2.lin_key.weight
torch.Size([256, 256])
convs.2.lin_key.bias
torch.Size([256])
convs.2.lin_query.weight
torch.Size([256, 256])
convs.2.lin_query.bias
torch.Size([256])
convs.2.lin_value.weight

1063937

In [5]:
lr

1e-05

In [ ]:
for i in enumerate(model.modules()):
    print(i)
#print("Shape of the output " + str(np.shape(model(train_data[0].x, edge_index, edge_attr=edge_attr))))

In [ ]:
test_datasets_lst = []
for i, graphdata in enumerate(graphdata_lst):
    run = wandb.init(
    # set the wandb project where this run will be logged
    project="OPF-GNN-Supervised-Homo",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Homo-GNN",
    "num_layers": num_layers,
    "dataset": grid_names[i],
    "epochs": 1000,
    "activation": activation,
    "dropout": dropout,
    "in_channels": in_channels,
    "output_channels": out_channels,
    "hidden_channels": hidden_channels,
    "channel type": layer_type,
    "norm": "BatchNorm",
    "scaler": scaler

    }
    )
    num_epochs = wandb.run.config.epochs
    run.watch(model)
    grid_name = graphdata.grid_name
    run.config.dataset = grid_name
    train_data = graphdata.train_data
    run.config["number of busses"] = np.shape(train_data[0].x)[0]
    val_data = graphdata.val_data
    test_data = graphdata.test_data
    test_datasets_lst.append(test_data)
    training_loader = DataLoader(train_data, batch_size=1, shuffle=True)
    validation_loader = DataLoader(val_data, batch_size=1, shuffle=True)
    #test_loader = DataLoader(test_data, batch_size=1, shuffle=True)
    for _ in range(num_epochs):
        #train_one_epoch(i, optimizer, training_loader, model, nn.MSELoss(), edge_index, edge_weights)
        train_validate_one_epoch(_, grid_name, optimizer, training_loader, validation_loader, model, nn.MSELoss(), scaler)
    print("Training and Validation finished " + "for GraphData " + str(i) + ".")


In [6]:
train_loader_lst = []
val_loader_lst = []
test_loader_lst = []
for i, graphdata in enumerate(graphdata_lst):

    # Divide training data into chunks, load into Dataloaders and append to the list of training loaders
    for train_data in divide_chunks(graphdata.train_data, 5):
        train_loader_lst.append(DataLoader(train_data, batch_size=1, shuffle=True))

    # Divide validation data into chunks, load into Dataloaders and append to the list of validation loaders
    for val_data in divide_chunks(graphdata.val_data, 5):
        val_loader_lst.append(DataLoader(val_data, batch_size=1, shuffle=True))

    # Append the test data to the list
    for test_data in divide_chunks(graphdata.test_data, 5):
        test_loader_lst.append(DataLoader(test_data, batch_size=1, shuffle=True))

print("Data Preparation finished.")

Data Preparation finished.


In [17]:
run = wandb.init(
    # set the wandb project where this run will be logged
    project="OPF-GNN-Supervised-Homo",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "Homo-GNN",
    "num_layers": num_layers,
    "epochs": 2000,
    "activation": activation,
    "dropout": dropout,
    "in_channels": in_channels,
    "output_channels": out_channels,
    "hidden_channels": hidden_channels,
    "channel type": layer_type,
    "norm": "BatchNorm",
    "scaler": scaler
    }
    )
for _ in range(wandb.run.config.epochs):
    # Training
    random.shuffle(train_loader_lst)
    print("Training the model for epoch " + str(_))
    train_all_one_epoch(_, optimizer, train_loader_lst, model, nn.MSELoss())

    # Validation
    random.shuffle(val_loader_lst)
    print("Validating the model for epoch " + str(_))
    validate_all_one_epoch(_, val_loader_lst, model, nn.MSELoss())
print("Training and Validation finished.")

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_mae_loss,▁
test_mre_loss,▁
test_rmse_loss,▁
train_mae_loss,▇█▆█▆▅▅▅▆▄▆▅▅▅▄▄▄▅▅▅▃▄▅▃▄▅▄▃▃▃▂▄▄▃▄▄▁▂▁▂
train_mre_loss,▆▇▅▆▅▅▅▄▄▄▆▇▆▅▄▅▄█▅▆▄▄▄▃▃▄▄▃▅▃▂▆▅▆▃▅▂▃▁▂
train_rmse_loss,▇█▇█▆▅▄▆▆▄▅▅▄▄▄▄▄▄▅▄▃▄▄▃▄▅▄▃▃▃▂▃▄▃▄▃▁▂▁▂
val_mae_loss,▇▃▄▆▂▄▄▄▄▆█▅▅▄▅▃▃▃▃▃▃▄▂▁▁▃▄▂▃▃▁▃▂▃▁▄▁▃▂▃
val_mre_loss,▅▃▄▄▂▁▂▂▃▅█▄▅▂▅▃▃▂▂▃▃▃▂▁▁▃▂▁▃▂▁▂▂▃▁▃▁▃▂▂
val_rmse_loss,█▃▄▇▂▅▄▄▄▆█▅▄▄▅▃▃▃▄▃▂▄▂▂▂▃▄▂▂▃▁▄▂▂▁▄▁▃▂▄
epoch,999


Training the model for epoch 0
Validating the model for epoch 0
Training the model for epoch 1
Validating the model for epoch 1
Training the model for epoch 2
Validating the model for epoch 2
Training the model for epoch 3
Validating the model for epoch 3
Training the model for epoch 4
Validating the model for epoch 4
Training the model for epoch 5
Validating the model for epoch 5
Training the model for epoch 6
Validating the model for epoch 6
Training the model for epoch 7
Validating the model for epoch 7
Training the model for epoch 8
Validating the model for epoch 8
Training the model for epoch 9
Validating the model for epoch 9
Training the model for epoch 10
Validating the model for epoch 10
Training the model for epoch 11
Validating the model for epoch 11
Training the model for epoch 12
Validating the model for epoch 12
Training the model for epoch 13
Validating the model for epoch 13
Training the model for epoch 14
Validating the model for epoch 14
Training the model for epoch 1

In [10]:
def test_all_one_epoch(test_loader_lst, model, loss_fn):

    test_rmse_loss = 0.0
    test_mae_loss = 0.0
    test_mre_loss = 0.0
    out = []

    # create a criterion to measure the mean absolute error
    mae_loss_fn = nn.L1Loss()

    # create a criterion to measure the mean relative error (MRE), outputs, targets : Torch.Tensor
    mre_loss_fn = lambda outputs, targets: get_mre_loss(outputs, targets)

    for i, test_loader in enumerate(test_loader_lst):

        # Here, we use enumerate(validation_loader) instead of
        # iter(validation_loader) so that we can track the batch
        # index and do some intra-epoch reporting
        for j, data in enumerate(test_loader):
            scaler = StandardScaler()
            inputs, targets = data.x, data.y

            # Get edge_index and edge_attr from DataLoader
            edge_index = test_loader.dataset[j].edge_index
            edge_attr = test_loader.dataset[j].edge_attr

            # Define Scaler and standardize inputs and targets
            targets = torch.tensor(scaler.fit_transform(targets), dtype=torch.float32)
            inputs = torch.tensor(scaler.transform(inputs), dtype=torch.float32)

            # Make predictions for this batch
            outputs = model(inputs, edge_index, edge_attr=edge_attr)

            # Compute the loss and its gradients
            rmse_loss = torch.sqrt(loss_fn(outputs, targets))

            # Compute MAE loss
            mae_loss = mae_loss_fn(outputs, targets)

            # Compute MRE loss
            mre_loss = mre_loss_fn(outputs, targets)

            # Gather data and report
            test_rmse_loss += rmse_loss.item()
            test_mae_loss += mae_loss.item()
            test_mre_loss += mre_loss.item()


            if j + 1 == len(test_loader.dataset):
                output = scaler.inverse_transform(outputs.detach().numpy())
                target = scaler.inverse_transform(targets.detach().numpy())
                out.append((output, target))

    num_samples = len(test_loader_lst) * len(test_loader_lst[0].dataset)
    rmse = test_rmse_loss / num_samples
    mae = test_mae_loss / num_samples
    mre = test_mre_loss / num_samples
    wandb.log({
        'test_rmse_loss': rmse,
        'test_mae_loss': mae,
        'test_mre_loss': mre
    })

    return out

In [19]:
outputs = test_all_one_epoch(test_loader_lst, model, nn.MSELoss())

In [26]:
#model.eval()
rmse_sum = 0.0
mae_sum = 0.0
mre_sum = 0.0
results = []

for test_data in test_datasets_lst:
    test_loader = DataLoader(test_data, batch_size=1, shuffle=True)
    output, target, rmse, mae, mre = test_one_epoch(test_loader, grid_name, model, nn.MSELoss(), scaler)
    results.append([output, target])
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"MRE: {mre}")

    rmse_sum += rmse
    mae_sum += mae
    mre_sum += mre

print("Testing finished.")
#n = len(test_datasets_lst)
#print(f"RMSE: {rmse_sum/n}")
#print(f"MAE: {mae_sum/n}")
#print(f"MRE: {mre_sum/n}")

RMSE: 0.8407899141311646
MAE: 0.49835764765739443
MRE: 1.8780303716659545
RMSE: 0.7570285201072693
MAE: 0.4469647169113159
MRE: 1.954324221611023
RMSE: 0.37624269127845766
MAE: 0.16541315019130706
MRE: 1.011414897441864
RMSE: 0.0034879735205322502
MAE: 0.0023037344217300415
MRE: 0.01825417634099722
Testing finished.


In [20]:
output,target = outputs[0]

In [21]:
output

array([[ 1.10014713e+00,  2.15122337e+01,  1.70148148e+02,
        -3.19718571e+01],
       [ 1.09265029e+00, -1.34326801e-01,  6.84861450e+02,
        -3.32083679e+02],
       [ 1.09968364e+00,  2.06156502e+01, -2.04842453e+01,
        -7.57575455e+01],
       [ 1.06722450e+00,  2.08271294e+01,  9.13921148e-02,
        -3.66050541e-01],
       [ 1.08435261e+00,  2.07464085e+01,  1.84899688e+00,
         1.11369252e+00],
       [ 1.01768291e+00,  1.12696362e+01,  3.97922873e+00,
         1.59521842e+00],
       [ 1.08647060e+00,  2.04298153e+01, -3.24569941e+00,
         8.49650085e-01],
       [ 1.08576310e+00,  2.06271114e+01, -1.83427925e+01,
         9.35626686e-01],
       [ 1.08374381e+00,  2.09187450e+01, -7.86230040e+00,
         1.12598395e+00],
       [ 1.01479840e+00,  1.13029528e+01,  7.75626421e+00,
         1.54393435e+01],
       [ 1.09270191e+00,  2.40568943e+01, -1.48405746e+02,
        -2.08457291e-01],
       [ 1.02974355e+00,  1.35930557e+01, -1.41636105e+01,
      

In [22]:
target

array([[ 1.09999990e+00,  2.16890373e+01,  1.69163132e+02,
        -3.28349342e+01],
       [ 1.09200001e+00,  1.34110451e-07,  6.87923462e+02,
        -3.32993347e+02],
       [ 1.09999990e+00,  2.05242043e+01, -1.99997311e+01,
        -7.50699997e+01],
       [ 1.06670272e+00,  2.09776993e+01,  1.49011612e-08,
        -5.40167093e-08],
       [ 1.08438468e+00,  2.06500626e+01,  2.76166940e+00,
         1.17982459e+00],
       [ 1.01820004e+00,  1.12786512e+01,  3.07390594e+00,
         1.07545924e+00],
       [ 1.08633983e+00,  2.03409195e+01, -2.87421513e+00,
         1.10506654e+00],
       [ 1.08562791e+00,  2.05326080e+01, -1.76669426e+01,
         1.07482457e+00],
       [ 1.08370268e+00,  2.08095589e+01, -6.66071320e+00,
         1.19255185e+00],
       [ 1.01521647e+00,  1.13202333e+01,  6.76881933e+00,
         1.49414282e+01],
       [ 1.09231937e+00,  2.42328434e+01, -1.49059998e+02,
        -5.40167093e-08],
       [ 1.03006494e+00,  1.35642233e+01, -1.46682043e+01,
      

In [28]:
path = r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\basemodel.pt" #os.path.dirname(os.path.abspath("gnn.ipynb")) + "\\Models\\Supervised\\" + "basemodel.pt" #r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised"
torch.save(model.state_dict(), "basemodel.pt")

In [25]:
r"C:\Users\canba\OneDrive\Masaüstü\OPFGNN\code\Models\Supervised\basemodel.pt"

'C:\\Users\\canba\\OneDrive\\Masaüstü\\OPFGNN\\code\\Models\\Supervised\\basemodel.pt'

In [5]:
num_epochs = 400#wandb.run.config.epochs
for i in range(num_epochs):
    #train_one_epoch(i, optimizer, training_loader, model, nn.MSELoss(), edge_index, edge_weights)
    train_validate_one_epoch(i, optimizer, training_loader, validation_loader, model, nn.MSELoss(), scaler)


Training the model for epoch 0


ValueError: Found indices in 'edge_index' that are larger than 63 (got 81). Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 64) in your node feature matrix and try again.

In [8]:
# Test the model on unseen test datasets
output, target = test_one_epoch(test_loader, model, nn.MSELoss(), scaler)

In [9]:
output

array([[ 1.0683101e+00,  2.3949530e-02,  8.8567574e+01, -8.4233574e+01],
       [ 1.0570036e+00,  5.8595318e-01,  9.7487521e-01,  4.5753181e-01],
       [ 1.0577838e+00,  6.0326773e-01,  1.0284468e+00,  3.0910528e-01],
       [ 1.0547656e+00,  9.4646263e-01,  3.7133813e+00,  1.9543473e+00],
       [ 1.0546682e+00,  9.7482544e-01,  1.0057598e+00,  3.4111780e-01],
       [ 1.0634830e+00,  2.0665102e+00, -8.4067173e+00,  4.3038779e-01],
       [ 1.0562860e+00,  1.0781766e+00,  9.6230876e-01,  3.8876796e-01],
       [ 1.0571203e+00,  1.1205333e+00, -9.9173832e+00,  3.2120639e-01],
       [ 1.0577351e+00,  5.9928018e-01,  9.9331748e-01,  3.3852065e-01],
       [ 1.0568979e+00,  2.5439677e+00, -3.2515743e+01,  3.4886771e-01],
       [ 1.0576197e+00,  6.1051732e-01,  1.0620753e+00,  3.9521188e-01],
       [ 1.0575702e+00,  6.2349463e-01, -8.4999676e+00,  6.3311143e+00],
       [ 1.0557477e+00,  1.8241770e+00, -1.2151316e+01,  3.3614326e-01],
       [ 1.0570177e+00,  6.4494389e-01,  1.0018415e

In [10]:
target

array([[ 1.0680000e+00,  2.6167893e-08,  8.8225975e+01, -8.4022377e+01],
       [ 1.0568820e+00,  5.9149092e-01,  9.8391211e-01,  4.0006351e-01],
       [ 1.0577081e+00,  6.0350513e-01,  1.0271353e+00,  3.7340641e-01],
       [ 1.0547794e+00,  9.4498694e-01,  3.7338138e+00,  1.9741520e+00],
       [ 1.0546870e+00,  9.7111952e-01,  1.0326620e+00,  3.8597685e-01],
       [ 1.0635955e+00,  2.0664642e+00, -8.4175396e+00,  3.8423419e-01],
       [ 1.0562855e+00,  1.0822675e+00,  9.4554377e-01,  3.9155012e-01],
       [ 1.0570624e+00,  1.1200924e+00, -9.8385983e+00,  3.7593150e-01],
       [ 1.0576770e+00,  6.0142428e-01,  9.5998025e-01,  3.6394203e-01],
       [ 1.0569530e+00,  2.5494852e+00, -3.2436195e+01,  3.7467927e-01],
       [ 1.0575235e+00,  6.1570406e-01,  1.0866743e+00,  4.1896653e-01],
       [ 1.0574749e+00,  6.2680441e-01, -8.4445477e+00,  6.4590898e+00],
       [ 1.0557743e+00,  1.8250703e+00, -1.2125282e+01,  3.7913626e-01],
       [ 1.0569822e+00,  6.4586937e-01,  1.0437330e

In [3]:
net = sb.get_simbench_net(grid_names[0])

In [11]:
import pandapower.topology as top
graph = top.create_nxgraph(net, calc_branch_impedances=True)
#graph.name = grid_names[0]

In [ ]:
from pandapower.plotting.simple_plot import simple_plot
from pandapower.plotting.plotly.simple_plotly import simple_plotly
#simple_plot(net, plot_gens=True, plot_loads=True, plot_sgens=True, library="igraph")
simple_plotly(net, map_style="satellite")